In [3]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import itertools
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import plotly.express as px
import plotly.graph_objects as go
import pickle
import time
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from scipy.spatial import distance
import dcor

In [4]:
pd.set_option('display.max_rows', 500)

#### load and preprocess data

In [5]:

data_df = pd.read_csv("country_indicators.csv",dtype={"Country Name": str, "Indicator Name": str, "Year": int, "Value": float})
indicators = data_df.groupby(['Country Name', 'Year'])['Indicator Name'].apply(lambda x: list(np.unique(x)))[0]
indicator_pairs = list(itertools.combinations(indicators, 2))
processed_df = data_df.groupby(['Country Name', 'Year', 'Indicator Name'])['Value'].aggregate('mean').unstack().reset_index()


## linear vs Spearman correlation

In [6]:
correlations = {}
for indicator_pair in indicator_pairs:
    correlations[indicator_pair] = {"Spearman": processed_df[indicator_pair[0]].corr(processed_df[indicator_pair[1]], method="spearman"), "Pearson": processed_df[indicator_pair[0]].corr(processed_df[indicator_pair[1]], method="pearson")}


In [7]:
correlations

{('Agriculture, value added (% of GDP)',
  'CO2 emissions (metric tons per capita)'): {'Spearman': -0.8466370427022761,
  'Pearson': -0.5547127404769792},
 ('Agriculture, value added (% of GDP)',
  'Domestic credit provided by financial sector (% of GDP)'): {'Spearman': -0.5575379914275259,
  'Pearson': -0.4598218268502184},
 ('Agriculture, value added (% of GDP)',
  'Electric power consumption (kWh per capita)'): {'Spearman': -0.827178927180471,
  'Pearson': -0.5055480816222381},
 ('Agriculture, value added (% of GDP)',
  'Energy use (kg of oil equivalent per capita)'): {'Spearman': -0.7995953902339793,
  'Pearson': -0.536290533085397},
 ('Agriculture, value added (% of GDP)',
  'Exports of goods and services (% of GDP)'): {'Spearman': -0.4792429043025526,
  'Pearson': -0.4133725267062382},
 ('Agriculture, value added (% of GDP)',
  'Fertility rate, total (births per woman)'): {'Spearman': 0.7145866572826729,
  'Pearson': 0.690861313963793},
 ('Agriculture, value added (% of GDP)',
  

## r2 of linear and non-linear regression

In [8]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [9]:
def rae(true, predicted):
    numerator = np.sum(np.abs(predicted - true))
    denominator = np.sum(np.abs(np.mean(true) - true))
    return numerator / denominator

In [10]:
# produce a shorter list of indicator tuples that we use for testing
index_list = [0,1,2,8,9,10,17]
routine_pairs = []
for i in index_list:
    routine_pairs.append(indicator_pairs[i])

In [19]:
# dict to save r2 and correlation values
r2_dict = {}
# dict to save x-/y-train/-test and predicted values
data_dict = {}

# dict for GridSearch
params = { "hidden_layer_sizes": [(12,), (50,), (70,5,)]}

start = time.time()

# run on all indicator_pairs or shorter selection in test runs
looping_pairs = routine_pairs
#looping_pairs = indicator_pairs

# to print the process of the routine
n=0

for curr_indicator_pair in looping_pairs:
    
    n+=1
    
    # reduce data to relevant columns and only then drop NaNs
    curr_data_no_nan = processed_df[[curr_indicator_pair[0], curr_indicator_pair[1]]].dropna()
    
    # get necessary arrays and do train test split
    curr_x = np.array(curr_data_no_nan[curr_indicator_pair[0]]).reshape(-1, 1)
    curr_y = np.array(curr_data_no_nan[curr_indicator_pair[1]])
    curr_X_train, curr_X_test, curr_y_train, curr_y_test = train_test_split(curr_x, curr_y, random_state=1, test_size=.1)
    
    # linear regression
    lin_reg = LinearRegression().fit(curr_X_train,curr_y_train)
    curr_lin_r2 = lin_reg.score(curr_X_test,curr_y_test)
    curr_y_test_pred = lin_reg.predict(curr_X_test)
    curr_lin_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
    curr_lin_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
    curr_lin_distcorr = distance.correlation(curr_y_test, curr_y_test_pred)
    curr_lin_rae = rae(curr_y_test, curr_y_test_pred)
    curr_lin_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)
    
    # non-linear MLP regression
    clf = GridSearchCV(estimator=MLPRegressor(max_iter=3000),
                       param_grid=params,
                       cv=2,
                       scoring='r2')
    clf.fit(curr_X_train, curr_y_train)
    curr_mlp_r2 = clf.score(curr_X_test, curr_y_test)
    curr_y_test_pred = clf.predict(curr_X_test)
    curr_mlp_rmse = mean_squared_error(curr_y_test, curr_y_test_pred, squared=False)
    curr_mlp_mape = mean_absolute_percentage_error(curr_y_test, curr_y_test_pred)
    curr_mlp_distcorr = distance.correlation(curr_y_test, curr_y_test_pred)
    curr_mlp_rae = rae(curr_y_test, curr_y_test_pred)
    curr_mlp_dcor = dcor.distance_correlation(curr_y_test, curr_y_test_pred)
    
    
    # save metrics into dict
    r2_dict[curr_indicator_pair] = {"MLP r2": curr_mlp_r2, "linear r2": curr_lin_r2,
                                    "MLP RMSE": curr_mlp_rmse, "linear RMSE": curr_lin_rmse,
                                    "MLP MAPE": curr_mlp_mape, "linear MAPE": curr_lin_mape,
                                    "MLP distcorr": curr_mlp_distcorr, "linear distcorr": curr_lin_distcorr,
                                    "MLP rae": curr_mlp_rae, "linear rae": curr_lin_rae,
                                    "MLP dcor": curr_mlp_dcor, "linear dcor": curr_lin_dcor,
        "Spearman": curr_data_no_nan[curr_indicator_pair[0]].corr(curr_data_no_nan[curr_indicator_pair[1]], method="spearman"), 
        "Pearson": curr_data_no_nan[curr_indicator_pair[0]].corr(curr_data_no_nan[curr_indicator_pair[1]], method="pearson")}
    
    # save values into dict
    data_dict[curr_indicator_pair] = {"X_train": curr_X_train, "X_test": curr_X_test, "y_train": curr_y_train,
                                     "y_test": curr_y_test, "y_test_pred": curr_y_test_pred}
    
    print(curr_indicator_pair)
    print("MLP r2: "+str(curr_mlp_r2)+"(layers: "+str(clf.best_params_["hidden_layer_sizes"])+")")
    print("linear r2: "+str(curr_lin_r2))
    print("Spearman corr: "+str(correlations[curr_indicator_pair]["Spearman"]))
    print("Pearson corr: "+str(correlations[curr_indicator_pair]["Pearson"]))
    print("----------"+str(n)+"/"+str(len(looping_pairs))+"-------------")

print("This took "+str(time.time()-start))

('Agriculture, value added (% of GDP)', 'CO2 emissions (metric tons per capita)')
MLP r2: 0.6417353823023969(layers: (70, 5))
linear r2: 0.43029080888803717
Spearman corr: -0.8466370427022761
Pearson corr: -0.5547127404769792
----------1/7-------------


/home/dw/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


('Agriculture, value added (% of GDP)', 'Domestic credit provided by financial sector (% of GDP)')
MLP r2: 0.21771616857834875(layers: (70, 5))
linear r2: 0.14796179710561008
Spearman corr: -0.5575379914275259
Pearson corr: -0.4598218268502184
----------2/7-------------
('Agriculture, value added (% of GDP)', 'Electric power consumption (kWh per capita)')
MLP r2: 0.43406063403298023(layers: (70, 5))
linear r2: 0.23323549566824353
Spearman corr: -0.827178927180471
Pearson corr: -0.5055480816222381
----------3/7-------------
('Agriculture, value added (% of GDP)', 'Industry, value added (% of GDP)')
MLP r2: 0.2880392157784528(layers: (50,))
linear r2: 0.2830703968527849
Spearman corr: -0.41874050640958005
Pearson corr: -0.45165236011360066
----------4/7-------------
('Agriculture, value added (% of GDP)', 'Inflation, GDP deflator (annual %)')
MLP r2: -0.010174641732326517(layers: (12,))
linear r2: -0.004857927401618722
Spearman corr: 0.18552184557197127
Pearson corr: 0.05962984524749174


In [14]:
r2_dict

{('Agriculture, value added (% of GDP)',
  'CO2 emissions (metric tons per capita)'): {'MLP r2': 0.6432114086672559,
  'linear r2': 0.43029080888803717,
  'MLP RMSE': 2.3079017936024675,
  'linear RMSE': 2.9163420387172927,
  'MLP MAPE': 1.6968343888926243,
  'linear MAPE': 6.154259022562997,
  'MLP distcorr': 0.18719623670664864,
  'linear distcorr': 0.3397546783314137,
  'MLP rae': 0.4993067919191389,
  'linear rae': 0.7342553696077134,
  'MLP dcor': 0.8317518026727804,
  'linear dcor': 0.7447676730891976,
  'Spearman': -0.8466370427022761,
  'Pearson': -0.5547127404769792},
 ('Agriculture, value added (% of GDP)',
  'Domestic credit provided by financial sector (% of GDP)'): {'MLP r2': 0.21651733305813203,
  'linear r2': 0.14796179710561008,
  'MLP RMSE': 44.55982792863173,
  'linear RMSE': 46.46846669768876,
  'MLP MAPE': 2.1091062935312372,
  'linear MAPE': 2.440726978011954,
  'MLP distcorr': 0.5339871963264158,
  'linear distcorr': 0.6142449193811381,
  'MLP rae': 0.843218231137

In [15]:
# to save / load the dict(s)

def save_r2_dict():
    with open('r2_dict.pkl', 'wb') as f:
        pickle.dump(r2_dict, f)

def load_r2_dict():
    with open('r2_dict.pkl', 'rb') as f:
        r2_dict = pickle.load(f)
    return r2_dict

def save_data_dict():
    with open('data_dict.pkl', 'wb') as f:
        pickle.dump(data_dict, f)

def load_data_dict():
    with open('data_dict.pkl', 'rb') as f:
        data_dict = pickle.load(f)
    return data_dict

In [20]:
#save_r2_dict()
#save_data_dict()

#r2_dict = load_r2_dict()
#data_dict = load_data_dict()

In [21]:
metrics_df = pd.DataFrame.from_dict(r2_dict).transpose()
metrics_df["|Spearman|"] = np.abs(metrics_df["Spearman"])
metrics_df["|Pearson|"] = np.abs(metrics_df["Pearson"])
metrics_df = metrics_df[["MLP r2", "linear r2", "MLP RMSE", "linear RMSE", "MLP MAPE", "linear MAPE", 
                         "MLP rae", "linear rae", "MLP distcorr", "linear distcorr",
                         "MLP dcor", "linear dcor", "|Spearman|", "|Pearson|", 
                         "Spearman", "Pearson"]].sort_values(by="MLP r2", ascending=False)


In [22]:
metrics_df

MLP r2  \
Electric power consumption (kWh per capita)        Energy use (kg of oil equivalent per capita)         0.823282   
Fertility rate, total (births per woman)           Life expectancy at birth, total (years)              0.729180   
Exports of goods and services (% of GDP)           Imports of goods and services (% of GDP)             0.708539   
Agriculture, value added (% of GDP)                CO2 emissions (metric tons per capita)               0.643211   
CO2 emissions (metric tons per capita)             Life expectancy at birth, total (years)              0.591550   
Agriculture, value added (% of GDP)                Population density (people per sq. km of land a...   0.587631   
CO2 emissions (metric tons per capita)             Fertility rate, total (births per woman)             0.581360   
Agriculture, value added (% of GDP)                Life expectancy at birth, total (years)              0.577497   
                                                   Fertility rate, total (births per woman)             0.532651   
                                                   Energy use (kg of oil equivalent per capita)         0.529110   
                                                   Services, etc., value added (% of GDP)               0.517301   
CO2 emissions (metric tons per capita)             Electric power consumption (kWh per capita)          0.447261   
Agriculture, value added (% of GDP)                Electric power consumption (kWh per capita)          0.418337   
Life expectancy at birth, total (years)            Services, etc., value added (% of GDP)               0.339434   
CO2 emissions (metric tons per capita)             Energy use (kg of oil equivalent per capita)         0.303834   
Agriculture, value added (% of GDP)                Industry, value added (% of GDP)                     0.291111   
Domestic credit provided by financial sector (%... Life expectancy at birth, total (years)              0.289532   
Energy use (kg of oil equivalent per capita)       Fertility rate, total (births per woman)             0.281954   
Agriculture, value added (% of GDP)                Exports of goods and services (% of GDP)             0.272126   
Domestic credit provided by financial sector (%... Services, etc., value added (% of GDP)               0.249518   
                                                   Electric power consumption (kWh per capita)          0.246011   
CO2 emissions (metric tons per capita)             Industry, value added (% of GDP)                     0.235624   
Domestic credit provided by financial sector (%... Fertility rate, total (births per woman)             0.230711   
CO2 emissions (metric tons per capita)             Services, etc., value added (% of GDP)               0.228406   
Agriculture, value added (% of GDP)                Domestic credit provided by financial sector (%...   0.216517   
Fertility rate, total (births per woman)           Services, etc., value added (% of GDP)               0.206291   
CO2 emissions (metric tons per capita)             Exports of goods and services (% of GDP)             0.148083   
                                                   Domestic credit provided by financial sector (%...   0.135666   
Imports of goods and services (% of GDP)           Population density (people per sq. km of land a...   0.124497   
Agriculture, value added (% of GDP)                Imports of goods and services (% of GDP)             0.121929   
Fertility rate, total (births per woman)           Industry, value added (% of GDP)                     0.111068   
Industry, value added (% of GDP)                   Life expectancy at birth, total (years)              0.101775   
Exports of goods and services (% of GDP)           Life expectancy at birth, total (years)              0.091479   
Domestic credit provided by financial sector (%... Energy use (kg of oil equivalent per capita)         0.088390   
Exports of goods and services (% of GDP)      

## Plotting (MLP) results

In [23]:
def plot_result(indicator_pair):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(x=data_dict[indicator_pair]["X_train"].reshape(len(data_dict[indicator_pair]["X_train"]),), 
                   y=data_dict[indicator_pair]["y_train"], 
                   name="y_train",
                   mode="markers", marker_color="Maroon", marker_size=3, opacity=.6
                    )
                 )
    fig.add_trace(
        go.Scatter(x=data_dict[indicator_pair]["X_test"].reshape(len(data_dict[indicator_pair]["X_test"]),), 
                   y=data_dict[indicator_pair]["y_test_pred"], 
                   name="y_pred",
                   mode="markers", marker_color="LightSeaGreen", 
                    )
                 )
    fig.add_trace(
        go.Scatter(x=data_dict[indicator_pair]["X_test"].reshape(len(data_dict[indicator_pair]["X_test"]),), 
                   y=data_dict[indicator_pair]["y_test"], 
                   name="y_test",
                   mode="markers", marker_color="LightSalmon", 
                    )
                 )

    fig.add_annotation(text='<b>r2 MLP:   </b>'+str(round(r2_dict[indicator_pair]["MLP r2"],2))+
                       ' <i><br>r2 lin. reg.:   </i>'+str(round(r2_dict[indicator_pair]["linear r2"],2))+
                       ' <b><br>RMSE MLP:   </b>'+str(round(r2_dict[indicator_pair]["MLP RMSE"],2))+
                       ' <i><br>RMSE lin. reg.:   </i>'+str(round(r2_dict[indicator_pair]["linear RMSE"],2))+
                       ' <b><br>MAPE MLP:   </b>'+str(round(r2_dict[indicator_pair]["MLP MAPE"],2))+
                       ' <i><br>MAPE lin. reg.:   </i>'+str(round(r2_dict[indicator_pair]["linear MAPE"],2))+
                       ' <b><br>rae MLP:   </b>'+str(round(r2_dict[indicator_pair]["MLP rae"],2))+
                       ' <i><br>rae lin. reg.:   </i>'+str(round(r2_dict[indicator_pair]["linear rae"],2))+
                       ' <b><br>dcor MLP:   </b>'+str(round(r2_dict[indicator_pair]["MLP dcor"],2))+
                       ' <i><br>dcor lin. reg.:   </i>'+str(round(r2_dict[indicator_pair]["linear dcor"],2))+
                       ' <br>Spearman corr.:   '+str(round(r2_dict[indicator_pair]["Spearman"],2))+
                       ' <br>Pearson corr.:   '+str(round(r2_dict[indicator_pair]["Pearson"],2)), 
                        align='right',
                        showarrow=False,
                        xref='paper',
                        yref='paper',
                        x=1.13,
                        y=.78,
                        bgcolor="white",
                        #bordercolor='black',
                        #borderwidth=1
                      )

    fig.update_layout(
        title=indicator_pair[1]+'  vs.  '+indicator_pair[0],
        xaxis=dict(
            title=indicator_pair[0],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        yaxis=dict(
            title=indicator_pair[1],
            gridcolor='white',
            gridwidth=2,
            #type='log',
        ),
        legend=dict(bgcolor="white"),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
    )

    fig.show()

In [42]:
indicator_pair = metrics_df.index[0]
indicator_pair

('Electric power consumption (kWh per capita)',
 'Energy use (kg of oil equivalent per capita)')

In [43]:
plot_result(indicator_pair)

In [50]:
np.std(data_dict[indicator_pair]["y_train"])

3759.5299098914934

In [46]:
data_dict[indicator_pair]["y_train"]

array([1.51137398e+02, 4.24711865e+02, 1.06011201e+03, 5.59699220e+02,
       4.42249077e+02, 5.90734931e+03, 7.70161213e+03, 1.15998866e+03,
       1.14402392e+04, 9.52665569e+03, 4.07174035e+03, 9.64060798e+03,
       1.51341042e+04, 3.07221163e+01, 2.77855247e+03, 7.57695613e+02,
       1.45598600e+03, 6.90078053e+03, 6.81160873e+01, 8.91805086e+02,
       3.38228639e+03, 2.32801463e+02, 4.72351682e+01, 6.95007399e+02,
       5.50165419e+03, 7.80125429e+02, 1.49697787e+02, 5.81910210e+03,
       1.71262027e+03, 1.90168979e+02, 1.59802768e+04, 7.13274514e+02,
       3.08041564e+02, 5.67378149e+01, 3.60094324e+02, 8.89349686e+01,
       4.00822659e+02, 4.03631449e+02, 6.72979416e+01, 3.84006963e+01,
       3.68525449e+04, 2.77255690e+02, 2.45960135e+02, 9.63030563e+02,
       7.67494070e+03, 1.29864801e+03, 9.00758786e+02, 1.08772251e+03,
       2.31867041e+02, 9.21349148e+02, 4.81159814e+03, 1.06012707e+02,
       4.50035230e+03, 5.64506029e+02, 3.08407573e+02, 2.48777160e+03,
      